In [92]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
%matplotlib inline


### Download data

First data is downloaded from github ad put into separated dataframes

In [93]:
covid = pd.read_json("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-json/dpc-covid19-ita-province.json")
provinces = pd.read_json("https://raw.githubusercontent.com/MatteoHenryChinaski/Comuni-Italiani-2018-Sql-Json-excel/master/italy_provincies.json")

covid = covid[(covid.sigla_provincia != "")].set_index("sigla_provincia")
provinces = provinces[provinces.sigla.notnull()].set_index("sigla")

### Join tables

Data is joined using the province code as index

In [94]:
df = covid.join(provinces, how="inner")

In [95]:
df["densita"] = df.apply(lambda row: float(row.residenti) / float(row.superficie), axis=1)

In [96]:
df.head()

,data,stato,codice_regione,denominazione_regione,codice_provincia,denominazione_provincia,lat,long,totale_casi,provincia,superficie,residenti,num_comuni,id_regione,densita
AG,2020-02-24 18:00:00,ITA,19,Sicilia,84,Agrigento,37.309711,13.584575,0,Agrigento,3044.85,446081,43,15.0,146.50344
AG,2020-02-25 18:00:00,ITA,19,Sicilia,84,Agrigento,37.309711,13.584575,0,Agrigento,3044.85,446081,43,15.0,146.50344
AG,2020-02-26 18:00:00,ITA,19,Sicilia,84,Agrigento,37.309711,13.584575,0,Agrigento,3044.85,446081,43,15.0,146.50344
AG,2020-02-27 18:00:00,ITA,19,Sicilia,84,Agrigento,37.309711,13.584575,0,Agrigento,3044.85,446081,43,15.0,146.50344
AG,2020-02-28 18:00:00,ITA,19,Sicilia,84,Agrigento,37.309711,13.584575,0,Agrigento,3044.85,446081,43,15.0,146.50344


In [115]:
FMT = '%Y-%m-%d %H:%M:%S'
BETA = 

def make_prediction(raw_df):
    df = raw_df[["data", "totale_casi", "densita", "num_comuni", "denominazione_provincia"]]    
    df["giorni"] = df.apply(lambda row: (datetime.strptime(row["data"], FMT) - datetime.strptime("2020-01-01 00:00:00", FMT)).days, axis=1)
    df = df.drop("data", axis=1)
    x = list(df.giorni)
    y = list(df.totale_casi)
    if (sum(df.totale_casi) < 10):
        return
    try:
        fit = curve_fit(logistic_model,x,y,p0=[2,100,20000], maxfev=10000)
        print(fit)
    except Exception as e:
        return


def logistic_model(x,a,b,c):
    return c/(1+np.exp(-(x-b)/a))


In [116]:
all_provinces = df.index.unique()
for province in all_provinces:
    data = df[df.index==province]
    make_prediction(data)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


(array([ 1.38243255, 65.37254148, 73.44869475]), array([[4.16433272e-02, 6.14804231e-02, 1.23171925e+00],
       [6.14804231e-02, 1.38469623e-01, 2.53634935e+00],
       [1.23171925e+00, 2.53634935e+00, 5.15217582e+01]]))
(array([ 1.53757547, 66.25705161, 84.2787515 ]), array([[1.48314231e-02, 3.21595784e-02, 7.31686431e-01],
       [3.21595784e-02, 8.80761573e-02, 1.97608691e+00],
       [7.31686431e-01, 1.97608691e+00, 4.59042944e+01]]))
(array([ 1.40021491, 66.58360162, 19.21143244]), array([[ 0.15861708,  0.362514  ,  2.09532265],
       [ 0.362514  ,  1.02667079,  5.88229455],
       [ 2.09532265,  5.88229455, 34.71583915]]))
(array([1.12920539e+00, 8.16873622e+01, 8.68863457e+05]), array([[1.22409015e+00, 3.95126529e+05, 3.01929048e+11],
       [3.95126529e+05, 1.36753706e+11, 1.04498228e+17],
       [3.01929048e+11, 1.04498228e+17, 7.98507039e+22]]))
(array([ 1.61522315, 65.10906927, 13.13704683]), array([[0.05923884, 0.09378856, 0.29216978],
       [0.09378856, 0.21709556, 0.63

(array([  4.35028424,  73.51831785, 532.18608493]), array([[4.45562914e-01, 4.45298957e+00, 3.76203834e+02],
       [4.45298957e+00, 4.78987771e+01, 4.10760952e+03],
       [3.76203834e+02, 4.10760952e+03, 3.53418123e+05]]))
(array([ 2.2291657 , 68.91552306, 70.59824691]), array([[6.18394128e-02, 3.34913428e-01, 6.02996031e+00],
       [3.34913428e-01, 1.99824526e+00, 3.66683980e+01],
       [6.02996031e+00, 3.66683980e+01, 6.77010715e+02]]))
(array([4.90462586e+00, 1.22116445e+02, 3.01440883e+06]), array([[1.21883337e+01, 1.35207805e+06, 8.27748876e+11],
       [1.35207805e+06, 1.63823384e+11, 1.00294359e+17],
       [8.27748876e+11, 1.00294359e+17, 6.14012375e+22]]))
(array([2.76761889e+00, 9.28231922e+01, 1.15714499e+05]), array([[1.29121415e+00, 1.79623747e+04, 7.50432118e+08],
       [1.79623747e+04, 2.77860466e+08, 1.16092379e+13],
       [7.50432118e+08, 1.16092379e+13, 4.85043489e+17]]))
(array([1.12314817e+01, 1.80373993e+02, 1.35340264e+05]), array([[1.70770605e+02, 5.8451623

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in exp


(array([ 1.9165137 , 67.77715627, 29.67163637]), array([[5.55444195e-02, 2.15193896e-01, 1.67948018e+00],
       [2.15193896e-01, 9.39145675e-01, 7.45266578e+00],
       [1.67948018e+00, 7.45266578e+00, 5.96742894e+01]]))
(array([3.00340460e+00, 9.56024523e+01, 2.62063112e+05]), array([[2.55445009e+00, 4.43917445e+04, 3.85739869e+09],
       [4.43917445e+04, 8.58228933e+08, 7.45795201e+13],
       [3.85739869e+09, 7.45795201e+13, 6.48091058e+18]]))
(array([  2.37266577,  61.54091975, 134.77135272]), array([[ 0.11326475,  0.12187935,  2.50352866],
       [ 0.12187935,  0.25334562,  4.23103418],
       [ 2.50352866,  4.23103418, 87.30570376]]))
(array([ 1.24093997, 63.99014893, 25.86016275]), array([[0.04158621, 0.03494403, 0.25265591],
       [0.03494403, 0.07829306, 0.39279647],
       [0.25265591, 0.39279647, 2.8518336 ]]))
(array([ 2.59561663, 67.72711717, 80.85689418]), array([[1.00410438e-01, 4.00448765e-01, 6.36234091e+00],
       [4.00448765e-01, 1.79721717e+00, 2.90696934e+01],
